In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
import os
import zipfile
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet101V2
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
datasets = "/content/drive/MyDrive/dataset.zip"
target_dir = 'data'  # Extract into Data Folder
os.makedirs(target_dir, exist_ok=True)

with zipfile.ZipFile(datasets, 'r') as zip_ref:
    zip_ref.extractall(target_dir)

In [4]:
breasttest = "/content/drive/MyDrive/breast/test"
chesttest = "/content/drive/MyDrive/chest/test"
malariatest = "/content/drive/MyDrive/malaria/test"

In [5]:
# Define data directory, target size, and batch size
data_dir = 'data'
target_size = (224, 224)
batch_size = 64

In [9]:
# Define function to load disease data
def load_disease_data(data_dir, disease_name, target_size, batch_size, validation_split=0.2):
    generator = ImageDataGenerator(rescale=1./255, validation_split=validation_split).flow_from_directory(
        os.path.join(data_dir, disease_name),
        target_size=target_size,
        batch_size=batch_size,
        classes=['Positive', 'Negative'],
        subset='training'
    )

    return generator

In [10]:
# Load disease data for chest_xrays, malaria, and breastcancer
chest_xrays_generator = load_disease_data(data_dir, 'chest_xray', target_size, batch_size)
malaria_generator = load_disease_data(data_dir, 'malariadataset', target_size, batch_size)
breastcancer_generator = load_disease_data(data_dir, 'breastcancer', target_size, batch_size)

Found 4173 images belonging to 2 classes.
Found 5645 images belonging to 2 classes.
Found 7721 images belonging to 2 classes.


In [11]:
# Define function to create disease-specific model
def create_disease_model(input_shape):
    base_model = ResNet101V2(include_top=False, weights='imagenet', input_shape=input_shape)
    x = base_model.output
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    predictions = Dense(2, activation='softmax')(x)  # Output two classes (positive/negative)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [12]:
# Create disease-specific models
chest_xrays_model = create_disease_model(target_size + (3,))
malaria_model = create_disease_model(target_size + (3,))
breastcancer_model = create_disease_model(target_size + (3,))

171317808/171317808 [==============================] - 1s 0us/step


In [13]:
# Define function for training models
def train_model(model, train_data, epochs):
    model.fit(train_data, epochs=epochs)

In [ ]:
# Train disease-specific models
train_model(chest_xrays_model, chest_xrays_generator, epochs=4)
train_model(malaria_model, malaria_generator, epochs=4)
train_model(breastcancer_model, breastcancer_generator, epochs=4)

Epoch 1/4
66/66 [==============================] - 120s 1s/step - loss: 0.5271 - accuracy: 0.8974
Epoch 2/4
66/66 [==============================] - 65s 974ms/step - loss: 0.1093 - accuracy: 0.9583
Epoch 3/4
66/66 [==============================] - 66s 999ms/step - loss: 0.2954 - accuracy: 0.9027
Epoch 4/4
66/66 [==============================] - 66s 996ms/step - loss: 0.1512 - accuracy: 0.9420
Epoch 1/4
89/89 [==============================] - 141s 986ms/step - loss: 0.7000 - accuracy: 0.8661
Epoch 2/4
89/89 [==============================] - 89s 994ms/step - loss: 0.1926 - accuracy: 0.9424
Epoch 3/4
89/89 [==============================] - 88s 988ms/step - loss: 0.1662 - accuracy: 0.9447
Epoch 4/4
89/89 [==============================] - 88s 983ms/step - loss: 0.2180 - accuracy: 0.9467
Epoch 1/4
121/121 [==============================] - 177s 1s/step - loss: 0.5595 - accuracy: 0.8315
Epoch 2/4
121/121 [==============================] - 123s 1s/step - loss: 0.2644 - accuracy: 0.8900
E

In [ ]:
chest_xrays_model.save('/content/drive/MyDrive/pneumonia_model.h5')
malaria_model.save('/content/drive/MyDrive/malaria_model.h5')
breastcancer_model.save('/content/drive/MyDrive/breastcancer_model.h5')

NameError: name 'chest_xrays_model' is not defined

In [ ]:
# Load trained models from Google Drive
from tensorflow.keras.models import load_model

pneumonia_model = load_model('/content/drive/MyDrive/pneumonia_model.h5')
malaria_model = load_model('/content/drive/MyDrive/malaria_model.h5')
breastcancer_model = load_model('/content/drive/MyDrive/breastcancer_model.h5')

In [ ]:
# Define function to load test data
def load_test_data(test_dir, target_size, batch_size):
    test_datagen = ImageDataGenerator(rescale=1./255)

    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=target_size,
        batch_size=batch_size,
        class_mode='categorical',  # Assuming binary classification (positive/negative)
        shuffle=False  # Keep the order of samples for evaluation
    )

    return test_generator

In [ ]:
# Example usage of load_test_data function
breast_test_generator = load_test_data("/content/drive/MyDrive/breast/test", target_size, batch_size)
chest_test_generator = load_test_data("/content/drive/MyDrive/chest/test", target_size, batch_size)
malaria_test_generator = load_test_data("/content/drive/MyDrive/malaria/test", target_size, batch_size)

Found 42 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 2756 images belonging to 2 classes.


In [ ]:
# Evaluate the models on test data
#pneumonia_eval = pneumonia_model.evaluate(chest_test_generator)
malaria_eval = malaria_model.evaluate(malaria_test_generator)
breastcancer_eval = breastcancer_model.evaluate(breast_test_generator)

18/44 [===========>..................] - ETA: 8:41 - loss: 0.0000e+00 - accuracy: 1.0000

In [ ]:
# Evaluate models on test set (if available)
if chest_xrays_test:
    chest_xrays_model.evaluate(chest_xrays_test)

if malaria_test:
    malaria_model.evaluate(malaria_test)

if breastcancer_test:
    breastcancer_model.evaluate(breastcancer_test)

In [ ]:
# Handle User-Uploaded Images for Inference
def predict_disease(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=target_size)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalize the image

    prediction = model.predict(img_array)
    # Process prediction and return result
    return prediction

In [ ]:
# Example usage for user-uploaded image
image_path = 'path_to_user_uploaded_image.jpg'
chest_xrays_prediction = predict_disease(image_path, chest_xrays_model)
malaria_prediction = predict_disease(image_path, malaria_model)
breastcancer_prediction = predict_disease(image_path, breastcancer_model)

FileNotFoundError: [Errno 2] No such file or directory: 'path_to_user_uploaded_image.jpg'

In [ ]:
print("Chest X-rays Prediction:", chest_xrays_prediction)
print("Malaria Prediction:", malaria_prediction)
print("Breast Cancer Prediction:", breastcancer_prediction)